In [3]:

!pip install sentence-transformers pandas openpyxl fuzzywuzzy gradio --quiet

from google.colab import drive
drive.mount('/content/drive')

file_path = "/content/drive/MyDrive/FAQ_1.xlsx"

import pandas as pd
from sentence_transformers import SentenceTransformer
import gradio as gr
import os
import torch


# FAQ

if os.path.exists(file_path):
    df = pd.read_excel(file_path)
    df = df.rename(columns={'Питання': 'question', 'Відповідь': 'answer'})
else:
    df = pd.DataFrame(columns=["question", "answer"])
    df.to_excel(file_path, index=False)


# NLP

# Model for creating embeddings (Russian/Ukrainian support)
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

# Update embeddings
def update_embeddings():
    global df, embeddings
    if len(df) > 0:
        embeddings = model.encode(df['question'].tolist(), convert_to_tensor=True)
    else:
        embeddings = None

update_embeddings()


# Chatbot functions

def get_answer(user_question):
    user_question = user_question.strip()


    greetings = ["привет", "добрый день", "доброго дня", "добрий день", "здрастуйте", "хай", "привіт"]
    farewells = ["пока", "до свидания", "до зустрічі", "всього доброго", "bye", "до побачення"]

    if any(g in user_question.lower() for g in greetings):
        return "Привіт! Чим я можу вам допомогти? "
    if any(f in user_question.lower() for f in farewells):
        return "До зустрічі!"


    if len(df) == 0:
        return "База пуста :("

# Creating a question embedding
    question_embedding = model.encode([user_question], convert_to_tensor=True)

    # Cos
    cos_sim = torch.nn.functional.cosine_similarity(question_embedding, embeddings)
    best_idx = torch.argmax(cos_sim).item()

    if cos_sim[best_idx] > 0.6:  # similarity threshold
        return df.iloc[best_idx]['answer']
    else:
        return "Я поки що не знаю відповіді. Додайте питання та відповдь, щоб я став краще!"

def add_qa(new_question, new_answer):
    global df
    new_question = new_question.strip()
    new_answer = new_answer.strip()
    if new_question and new_answer:
        df = pd.concat([df, pd.DataFrame([[new_question, new_answer]], columns=['question', 'answer'])], ignore_index=True)
        df.to_excel(file_path, index=False)
        update_embeddings()
        return "Питання та відповідь доданні до бази!"
    else:
        return "Заповніть обидва поля, будь ласка"


# Gradio

with gr.Blocks() as demo:
    gr.Markdown("## Банківский чат-бот (RU/UA)")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Ваше питання")
    with gr.Row():
        submit = gr.Button("Надіслати")

    with gr.Accordion("Додати нове запитання", open=False):
        new_question = gr.Textbox(label="Нове запитання")
        new_answer = gr.Textbox(label="Нова відповідь")
        add_btn = gr.Button("Додати до бази")
        add_output = gr.Textbox(label="Статус")


    def respond(message, chat_history):
        answer = get_answer(message)
        chat_history.append((message, answer))
        return chat_history, ""

    submit.click(respond, inputs=[msg, chatbot], outputs=[chatbot, msg])
    add_btn.click(add_qa, inputs=[new_question, new_answer], outputs=[add_output])

demo.launch()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-3590198812.py:89: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9a0e7ea9e1b131316f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
